In [1]:
import urllib.request
urllib.request.urlretrieve("https://huggingface.co/datasets/hugginglearners/amazon-reviews-sentiment-analysis/resolve/main/amazon_reviews.csv?download=true", "amazon_reviews.csv")

('amazon_reviews.csv', <http.client.HTTPMessage at 0x7f83c1be0b90>)

In [2]:
!pip install pandas --quiet

In [3]:
import pandas as pd

reviews = pd.read_csv("amazon_reviews.csv")

In [4]:
#stratified sample
SAMPLES_PER_SCORE = 10
seed_selection_df = reviews.groupby('overall', group_keys=False).apply(lambda x: x.sample(min(len(x), SAMPLES_PER_SCORE)))

/tmp/ipykernel_303/1479949678.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  seed_selection_df = reviews.groupby('overall', group_keys=False).apply(lambda x: x.sample(min(len(x), SAMPLES_PER_SCORE)))


In [5]:
seed_selection_df.describe()

,Unnamed: 0,overall,day_diff,helpful_yes,helpful_no,total_vote,score_pos_neg_diff,score_average_rating,wilson_lower_bound
count,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000
mean,2537.640000,3.000000,456.220000,0.240000,0.280000,0.520000,-0.040000,0.099333,0.026621
std,1269.304144,1.428571,212.383201,0.686904,0.671277,1.110984,0.781417,0.252290,0.078227
min,319.000000,1.000000,129.000000,0.000000,0.000000,0.000000,-3.000000,0.000000,0.000000
25%,1642.500000,2.000000,247.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2420.500000,3.000000,477.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,3489.000000,4.000000,633.750000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,4770.000000,5.000000,910.000000,4.000000,3.000000,5.000000,3.000000,1.000000,0.375535


In [36]:
seed_selection_df.head()

,Unnamed: 0,reviewerName,overall,reviewText,reviewTime,day_diff,helpful_yes,helpful_no,total_vote,score_pos_neg_diff,score_average_rating,wilson_lower_bound
1866,1866,"Gregory P. Alford ""Senior Consultant""",1.0,This is one of those basic items that usually ...,2013-05-02,585,1,1,2,0,0.500000,0.094531
1593,1593,Eric Redman,1.0,Could not figure out why this card crashed mul...,2012-12-14,724,1,2,3,-1,0.333333,0.061492
1887,1887,"G. Stucco ""mr guido""",1.0,Broke after 2 weeks.....I will go back to Sam...,2013-07-12,514,1,0,1,1,1.000000,0.206549
3126,3126,M,1.0,I had it for a day before it died and lost all...,2014-06-01,190,0,0,0,0,0.000000,0.000000
593,593,bgcam,1.0,I was better off with my 32gig card. This thi...,2013-04-24,593,0,0,0,0,0.000000,0.000000


In [7]:
def translate_score_to_sentiment(score: int) -> str:
    normalized_score = int(score)-1
    if normalized_score < 5:
        return ["Strong Negative", "Mild Negative", "Neutral", "Mild Positive", "Strong Positive"][normalized_score]

In [16]:
%cd .. 

/home/jovyan/work


In [17]:
import json
import random

instances = []
for i, row in seed_selection_df.iterrows():
    instances.append({
        "id": f"seed_task_{i}", 
        "name": "sentiment_analysis", 
        "ticket": row["reviewText"], 
        "instances": [
            {
                "input": "", 
                "output": translate_score_to_sentiment(row["overall"])
            }
        ],
        "is_classification": False
    })

with open("./stanford_alpaca/seed_tasks.jsonl", "w") as outfile:
    for instance in instances:
        json.dump(instance, outfile)
        outfile.write('\n')

In [2]:
import os
import getpass

if not os.environ["OPENAI_API_KEY"]:
    os.environ["OPENAI_API_KEY"] = getpass.getpass()

In [18]:
%cd stanford_alpaca 

/home/jovyan/work/stanford_alpaca


In [35]:
%%bash
rm regen.json
python -m generate_instruction generate_instruction_following_data \
  --output_dir ./ \
  --num_instructions_to_generate 1000 \
  --model_name="gpt-4o-mini"

Loaded 50 human-written seed instructions


prompt_batches: 100%|██████████| 1/1 [00:53<00:00, 53.64s/it]


Request 1 took 53.64s, processing took 3.89s
Generated 73 instructions, kept 73 instructions


prompt_batches: 100%|██████████| 1/1 [01:33<00:00, 93.81s/it]


Request 2 took 93.81s, processing took 3.80s
Generated 62 instructions, kept 62 instructions


...

Request 12 took 80.89s, processing took 10.29s
Generated 77 instructions, kept 77 instructions


prompt_batches: 100%|██████████| 1/1 [01:26<00:00, 86.61s/it]


Request 13 took 86.61s, processing took 9.21s
Generated 74 instructions, kept 74 instructions


prompt_batches: 100%|██████████| 1/1 [01:42<00:00, 102.55s/it]
1008it [26:24,  1.57s/it]                          


Request 14 took 102.56s, processing took 10.51s
Generated 76 instructions, kept 76 instructions
